# Category-specific functional coupling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wikisim import rsa

roi_dir = '/Users/morton/Dropbox/work/bender/batch/glm/prex_stim2/roi'

## Calculate trial correlations for people and places

Calculate mean activation in each ROI for each trial, then calculate the correlation between each pair of ROIs during face trials and scene trials separately.

In [10]:
# get ROIs needed for the analysis
func_rois = rsa.get_rois('func')
pmat_rois = rsa.get_rois('pmat')
all_rois = pd.concat((func_rois, pmat_rois), axis=0)
rois = all_rois.loc[all_rois['net'].isin(['face', 'scene', 'hpc'])].copy()
rois

,roi_label,net,net_label,net_id
r_ofa,OFA,face,Face,0
b_ffa,FFA,face,Face,0
b_pat,ATFA,face,Face,0
b_opa,OPA,scene,Scene,1
b_rsc,RSC,scene,Scene,1
b_ppa,PPA,scene,Scene,1
b_fshs_phpc,pHPC,hpc,HPC,1
b_fshs_ahpc,aHPC,hpc,HPC,1


In [11]:
# calculate correlations
corr = rsa.load_roi_corr(roi_dir, rois.index)

## Test for coupling differences between people and place trials

In [7]:
# for correlations with pHPC and aHPC, calculate face - scene
func_rois = rois['roi_label'][:6]
df = {}
diff = corr['face'][:, :, 6:] - corr['scene'][:, :, 6:]
for i, roi in enumerate(func_rois):
    x = diff[:, i, :]
    df[roi] = pd.DataFrame(x, columns=['pHPC', 'aHPC'])

In [20]:
# test direction depends on whether the ROI represented person or place semantics
sig_rois = ['ATFA', 'OFA', 'OPA', 'RSC', 'PPA']
sign_rois = [1, -1, -1, -1, -1]
for roi, sign in zip(sig_rois, sign_rois):
    p, p_cor = rsa.sign_perm(
        df[roi].to_numpy() * sign, 100000, method='fwe', tail='right'
    )
    temp = df[roi] * sign
    st = temp.agg(['mean', 'std', 'sem'])
    st.loc['d', :] = np.abs(st.loc['mean']) / st.loc['std']
    st.loc['p', :] = p_cor
    print(roi)
    display(st)
    print()

ATFA


,pHPC,aHPC
mean,0.030864,0.068864
std,0.195168,0.139726
sem,0.033974,0.024323
d,0.158141,0.492852
p,0.255147,0.024100



OFA


,pHPC,aHPC
mean,-0.065640,-0.012973
std,0.140248,0.196903
sem,0.024414,0.034276
d,0.468030,0.065883
p,0.999130,0.822292



OPA


,pHPC,aHPC
mean,-0.037188,0.023016
std,0.161410,0.216836
sem,0.028098,0.037746
d,0.230394,0.106143
p,0.948861,0.361526



RSC


,pHPC,aHPC
mean,0.017136,0.090630
std,0.166996,0.168908
sem,0.029070,0.029403
d,0.102610,0.536562
p,0.434716,0.002890



PPA


,pHPC,aHPC
mean,-0.013653,0.065875
std,0.155910,0.179866
sem,0.027140,0.031311
d,0.087568,0.366247
p,0.838832,0.027120
